# Tensorflow Estimator Tutorial
---
## 이 문서의 내용:
AWS Sagemaekr에서 Tensorflow를 사용하려면 Estimator API 형태로 코드 작성이 필요합니다.  
이번 튜토리얼은 Sagemaker를 사용하기 전에 Estimator를 사용하는 것에 대해서 간단하게 살펴 볼 예정입니다.

## Estimator api 특징
Estimator를 사용해서 머신러닝 코드를 작성할시 Estimator는 아래와 같은 작업을 해줍니다.  
 - build the graph
 - initialize variables
 - start queues
 - handle exceptions
 - create checkpoint files and recover from failures
 - save summaries for TensorBoard


### Estimator 구성
---
```python
# 어떤 input(feature)이 neural net에 전달될지 
feature_columns = [tf.feature_column.numeric_column("inputs", shape=[2])]

# nerual net 구성
classifier = tf.estimator.DNNClassifier()

# train or evaluation 시 사용할 input function (위에서 정의한 feature column과 동일한 key 값을 가진 data를 return해 주어야 한다.)
train_input_fn = tf.estimator.inputs.numpy_input_fn()
eval_input_fn = tf.estimator.inputs.numpy_input_fn()

```

### xor input data
---
XOR 문제에 대한 input 값은 아래 4개 입니다.

```
0 ^ 0 = 0
0 ^ 1 = 1
1 ^ 0 = 1
1 ^ 1 = 0
``` 

### directory 구조는 아래와 같습니다.

In [ ]:
!tree

### Load data-set   
---
학습에 사용할 데이터를 읽어 옵니다.

In [ ]:
import tensorflow as tf
import sagemaker
import numpy as np

train_set = np.loadtxt('data/xor_train.csv', delimiter=',')
test_set = np.loadtxt('data/xor_test.csv', delimiter=',')
print(train_set)

### Deep neural network 구성
---
Estimator api는 predefined model을 가지고 있습니다. 이번에는 그중 *DNNClassifier*를 사용해보겠습니다.  
(Custom model도 Estimator로 작성 및 사용 가능함)

In [ ]:
# 학습에 사용될 feature
feature_columns = [tf.feature_column.numeric_column("inputs", shape=[2])]

# 10 x 20 x 10 의 neural net 구성
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                        hidden_units=[10, 20, 10],
                                        n_classes=2,
                                        model_dir="model",
                                        config=None)

`feature_columns=feature_columns.` XOR의 결과는 2개의 binary 값으로 결정 되므로 type은 numeric value, shpae는 [2]로 정의했습니다.   


`hidden_units=[10, 20, 10]` 3개의 hidden layer를 설정했습니다.    


`n_classes=2` 결과는 0또는 1 이므로 2로 값을 설정 했습니다.  


`model_dir=model` checkpoint data 와 TensorBoard summaries 가 저장되는 위치  


`config` None으로 할시 default config가 적용된다.  


config 에서는 checkpoint값을 시간 or step별로 주기를 지정하거나 cluster spec등을 설정해줄 수 있습니다.

### Input function 작성
---
*tf.estimator* api는 input function을 사용합니다.  
input pipeline을 작성을 위해서 *tf.estimator.inputs.numpy_input_fn* 을 사용하겠습니다.

In [ ]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"inputs": train_set[:, 0:-1]},
    y=np.array(train_set[:, [-1]]),
    num_epochs=None,
    shuffle=True)

### 학습
---

In [ ]:
classifier.train(input_fn=train_input_fn, steps=1000)

### 평가
---

In [ ]:
# Define the training inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"inputs": test_set[:, 0:-1]},
    y=np.array(test_set[:, [-1]]),
    num_epochs=10,
    shuffle=True)

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))